In [1]:
import pandas as pd
import re

# import csv files as Pandas dataframes

In [2]:
# geologiclog_quickpick ; released by Ben Brezing on OpenData/OSWCR 09/26/2024
df = pd.read_csv("geologiclog_quickpick.csv")
df.head()

,WCRNUMBER,INTERVALSTART,INTERVALEND,MATERIALTYPE,MATERIALCOLOR,MATERIALTEXTURE,MATERIALDESCRIPTION
0,WCR2019-016089,230.0,236.0,Gravel,Light Brown,Medium To Coarse,NaN
1,WCR2019-016089,236.0,245.0,Clay,Brown,Dense,NaN
2,WCR2019-016089,245.0,250.0,Gravel,Brown,Coarse,NaN
3,WCR2019-016089,250.0,256.0,Clayey Sand,Brown,Silty,NaN
4,WCR2019-016089,256.0,265.0,Gravel,Black,Coarse,NaN


In [3]:
keywords_keep = {
    # Compound soil types
    'sand/clay', 'sand & clay', 'clay/sand', 'clay & sand', 'clay/silt', 'clay & silt','cobble',
    'gravel/sand', 'gravel & sand', 'sand/gravel', 'sand & gravel', 'silty gravel',
    'sand/gravel/clay', 'gravel/sand/silt', 'silty clay', 'decomposed granite', 'sand/silt', 
    'sandy silt', 'gravelly clay', 'gravelly sand', 'silty clayey sand', 'sandy silt clay', 'gravelly silt',
    'gravelly silty sand', 'clayey sand', 'silty sand', 'gravel/silt', 'organic gravel', 'gravelly peat',
    'gravel/clay', 'gravelly loam', 'sandy loam', 'clayey loam', 'silty loam', 'gravelly clayey sand', 
    'organic clay', 'organic silt', 'organic loam', 'peaty soil', 'peaty silt', 'sand/clay/silt','sandstone',
    'clay loam', 'sandy clay loam', 'silt loam', 'silty gravel/sand', 'clay/gravel','clay/sand/gravel',
  
    # Sedimentary rocks
    'graywacke', 'conglomerate', 'shale', 'siltstone', 'limestone', 'cobblestone',  'mudstone', 'claystone',
    
    # Soil classifications
    'silt', 'sand',  'gravel',  'clay', 'boulder', 

    # Soil descriptors
    'sticky', 'fat', 'lean', 'hardpan', 'organic', 'adobe',  'weathered','poorly graded sand', 'well graded sand',
    'poorly graded gravel', 'well graded gravel','asphalt',
    # Rocks and minerals
    'basalt', 'basaltic', 'pumice', 'latite', 'volcanic', 'volcanics', 'cinder', 'ash', 
    'tufa', 'tuff', 'lava', 'rhyolite', 'granite', 'diorite', 'quartz', 'gabbro', 'quartzite', 
    'granodiorite', 'igneous', 'andesite', 'greenstone', 'Slate', 'schist', 'serpentine', 
    'metasediment', 'phyllite', 'argillite', 'bluestone', 'soapstone', 'chert','fractured',
    'fractured rock', 'gouge', 'hardrock', 'rock','bedrock','frac','tuscan','lapilli','limestone',

    # Other geological terms
    'sediment', 'alluvium', 'loam', 'peat', 'organics','topsoil','soil','Other-Coarse Grained','Other-Fine Grained',
}

In [4]:
# Define a function to concatenate only non-NaN values
def concat_non_nan(row):
    # Concatenate only if the values are not NaN
    if pd.notna(row['MATERIALTYPE']) and pd.notna(row['MATERIALTEXTURE']) and pd.notna(row['MATERIALDESCRIPTION']):
        return f"{row['MATERIALTYPE']} {row['MATERIALTEXTURE']} {row['MATERIALDESCRIPTION']}"
    else:
        return row['MATERIALTYPE'] # Return None if any of the values is NaN

# Apply the function to create the new concatenated column
df['COMBINED_MATERIALS'] = df.apply(concat_non_nan, axis=1)

In [5]:
# Select only the key words to convert into USCS
descrip = df['COMBINED_MATERIALS']

# Build the regex pattern
pattern = r'\b(?:' + '|'.join(map(re.escape, keywords_keep)) + r')\b'

# Apply the regex, handling NaN values by converting them to an empty string
extracted_words = [re.findall(pattern, str(d), re.IGNORECASE) for d in descrip]

# Add the extracted words as a new column
df['KEYWORDS_LIST'] = extracted_words

# Join the keywords into a single string
new_descriptions = df['KEYWORDS_LIST']
separator = ','
d = []

for new_description in new_descriptions:
    new_descrip = separator.join(new_description)
    d.append(new_descrip.lower())

# Add the final keywords column
df['KEYWORDS'] = d

# Drop the intermediate column
df = df.drop(['KEYWORDS_LIST'], axis=1)

# Display the updated DataFrame
df.head()

,WCRNUMBER,INTERVALSTART,INTERVALEND,MATERIALTYPE,MATERIALCOLOR,MATERIALTEXTURE,MATERIALDESCRIPTION,COMBINED_MATERIALS,KEYWORDS
0,WCR2019-016089,230.0,236.0,Gravel,Light Brown,Medium To Coarse,NaN,Gravel,gravel
1,WCR2019-016089,236.0,245.0,Clay,Brown,Dense,NaN,Clay,clay
2,WCR2019-016089,245.0,250.0,Gravel,Brown,Coarse,NaN,Gravel,gravel
3,WCR2019-016089,250.0,256.0,Clayey Sand,Brown,Silty,NaN,Clayey Sand,clayey sand
4,WCR2019-016089,256.0,265.0,Gravel,Black,Coarse,NaN,Gravel,gravel


In [6]:
# Define a dictionary for USCS group symbols
uscs_conversions = {
    'Silty Sand': 'SM',
    'Sand': 'SW',
    'Clay': 'CH',
    'Concrete':'unknown',
    'Silt': 'ML',
    'Clayey Sand': 'SC',
    'Asphalt':'ASPT',
    'Soil or Organic': 'TPSL, OH',
    'soil,organic': 'TPSL, OH',
    'Granite': 'GRNT',
    'quartz': 'GRNT',
    'Shale': 'SHLE',
    'Rock': 'ROCK',
    'claystone': 'CLSN',
    'musdtone': 'CLSN',
    'Sandstone': 'SDST',
    'Gravel': 'GW',
    'Clayey Gravel': 'GC',
    
    'Silty Gravel': 'GM',
    'Serpentine':'META',
    'Greenstone':'META',
    'Unknown':'unknown',
    'Conglomerate': 'CONG',
    'Siltstone': 'STST',
    'Slate':'SCHT',
    'Schist':'SCHT',
    'Other-Coarse Grained' :'GW-SW',
    'Other-Fine Grained':'CH-ML',
    
    
    # Add more USCS conversions as needed
}

In [7]:
# Function to assign USCS group symbols based on Category (case-insensitive)
def assign_uscs(text, uscs_conversions):
    # Check if text is a string to avoid applying regex to NaNs or non-string types
    if not isinstance(text, str):
        return 'unknown'
    
    for key, value in uscs_conversions.items():
        # Use re.IGNORECASE for case-insensitive matching
        if re.search(r'\b{}\b'.format(re.escape(key)), text, re.IGNORECASE):
            return value
    return 'unknown'

In [8]:
# Apply the assign_uscs function to the 'COMBINED_MATERIALS' column (case-insensitive)
df['USCS'] = df['KEYWORDS'].apply(lambda x: assign_uscs(x, uscs_conversions))

# Print the updated DataFrame
df.head()

,WCRNUMBER,INTERVALSTART,INTERVALEND,MATERIALTYPE,MATERIALCOLOR,MATERIALTEXTURE,MATERIALDESCRIPTION,COMBINED_MATERIALS,KEYWORDS,USCS
0,WCR2019-016089,230.0,236.0,Gravel,Light Brown,Medium To Coarse,NaN,Gravel,gravel,GW
1,WCR2019-016089,236.0,245.0,Clay,Brown,Dense,NaN,Clay,clay,CH
2,WCR2019-016089,245.0,250.0,Gravel,Brown,Coarse,NaN,Gravel,gravel,GW
3,WCR2019-016089,250.0,256.0,Clayey Sand,Brown,Silty,NaN,Clayey Sand,clayey sand,SW
4,WCR2019-016089,256.0,265.0,Gravel,Black,Coarse,NaN,Gravel,gravel,GW


In [9]:
# Load the provided Excel file
file_path = r'C:\Users\betebari\Documents\C2VSim_Texture\OSWCR\USCS-averageKxy-CoarseFractions.xlsx'
excel_data = pd.read_excel(file_path)

# Strip any leading/trailing spaces and remove dashes from 'Sediment/Rock Type' column
excel_data['Sediment/Rock Type'] = excel_data['Sediment/Rock Type'].str.strip().str.replace('-', ' ', regex=False)

# Convert 'Sediment/Rock Type' to lowercase for case-insensitive matching
excel_data['Sediment/Rock Type'] = excel_data['Sediment/Rock Type'].str.lower()

# Create a dictionary mapping Soil Classification to Average Hydraulic Conductivity (case-insensitive)
hydraulic_conductivity_mapping = dict(zip(excel_data['Sediment/Rock Type'], excel_data['Average Hydraulic Conductivity (ft/day)']))

# Create a dictionary mapping Soil Classification to Average Coarse Fraction (case-insensitive)
coarse_fraction_mapping = dict(zip(excel_data['Sediment/Rock Type'], excel_data['Average Coarse Fraction (%)']))

# Strip any leading/trailing spaces and remove dashes from 'USCS' column in the merged DataFrame
df['USCS'] = df['USCS'].str.strip().str.replace('-', ' ', regex=False)

# Remove parentheses from USCS values
df['USCS'] = df['USCS'].str.replace(r'[\(\)]', '', regex=True)

# Convert 'USCS' column to lowercase for case-insensitive matching
df['USCS'] = df['USCS'].str.lower()

# Create separate mappings for Specific Yield, Ss, and Kv based on the Excel data columns
specific_yield_mapping = dict(zip(excel_data['Sediment/Rock Type'], excel_data['Avg Specific Yield (%)']))
ss_mapping = dict(zip(excel_data['Sediment/Rock Type'], excel_data['Avg Ss (1/L)']))
kv_mapping = dict(zip(excel_data['Sediment/Rock Type'], excel_data['Avg Kv (ft/day)']))

# Updated function to handle USCS combinations and unknown values
def aggregate_uscs_values(uscs_value, mapping, agg_func='average'):
    # Handle 'unknown' case by returning None or a default value
    if 'unknown' in uscs_value:
        return None  # You could replace None with a default value like 0 or 12 depending on your needs
    
    # Split combined USCS values (e.g., 'gw sw') into individual classifications
    uscs_list = [item.strip().lower() for item in uscs_value.split(' ')]
    
    values = [mapping.get(uscs) for uscs in uscs_list if uscs in mapping]
    
    # Apply 12% coarse fraction if secondary USCS classification is present and the 'coarse_fraction_mapping' is missing
    if 'gc' in uscs_list or 'sc' in uscs_list or 'gm' in uscs_list or 'sm' in uscs_list:
        if agg_func == 'average':
            # Assign 12% when coarse fraction data is not found for the given classification
            return sum(values) / len(values) if values else 12
        elif agg_func == 'max':
            return max(values) if values else 12
    
    if values:
        if agg_func == 'average':
            return sum(values) / len(values)
        elif agg_func == 'max':
            return max(values)
    return None

# Apply the aggregation function for Hydraulic Conductivity and Coarse Fraction (case-insensitive)
df['HydraulicConductivity'] = df['USCS'].apply(lambda x: aggregate_uscs_values(x, hydraulic_conductivity_mapping, agg_func='average'))
df['AverageCoarseFraction'] = df['USCS'].apply(lambda x: aggregate_uscs_values(x, coarse_fraction_mapping, agg_func='average'))
df['Avg Specific Yield (%)'] = df['USCS'].apply(lambda x: aggregate_uscs_values(x, specific_yield_mapping, agg_func='average'))
df['Avg Ss (1/L)'] = df['USCS'].apply(lambda x: aggregate_uscs_values(x, ss_mapping, agg_func='average'))
df['Avg Kv (ft/day)'] = df['USCS'].apply(lambda x: aggregate_uscs_values(x, kv_mapping, agg_func='average'))

# Identify and display any unmatched values
unmatched_values = df[df['HydraulicConductivity'].isna()]['USCS'].unique()
print("Unmatched 'USCS' values:", unmatched_values)

# Clean data
if 'Unnamed: 0' in df.columns:
    df = df.drop(['Unnamed: 0'], axis=1)

# Display the first few rows of the merged dataframe
df.head()


Unmatched 'USCS' values: ['unknown']


,WCRNUMBER,INTERVALSTART,INTERVALEND,MATERIALTYPE,MATERIALCOLOR,MATERIALTEXTURE,MATERIALDESCRIPTION,COMBINED_MATERIALS,KEYWORDS,USCS,HydraulicConductivity,AverageCoarseFraction,Avg Specific Yield (%),Avg Ss (1/L),Avg Kv (ft/day)
0,WCR2019-016089,230.0,236.0,Gravel,Light Brown,Medium To Coarse,NaN,Gravel,gravel,gw,167.519700,80.0,24.0,0.000051,0.5000
1,WCR2019-016089,236.0,245.0,Clay,Brown,Dense,NaN,Clay,clay,ch,0.016402,2.5,2.0,0.003490,0.0005
2,WCR2019-016089,245.0,250.0,Gravel,Brown,Coarse,NaN,Gravel,gravel,gw,167.519700,80.0,24.0,0.000051,0.5000
3,WCR2019-016089,250.0,256.0,Clayey Sand,Brown,Silty,NaN,Clayey Sand,clayey sand,sw,4.510000,60.0,35.0,0.000230,0.5000
4,WCR2019-016089,256.0,265.0,Gravel,Black,Coarse,NaN,Gravel,gravel,gw,167.519700,80.0,24.0,0.000051,0.5000


In [10]:
df1 = pd.read_csv("UPDATED_wellcompletionreports.csv")
# Merge df1 and df2 on 'WCRNUMBER'
merged_df = pd.merge(df1, df, on='WCRNUMBER', how='inner')

# Drop rows where 'DESCRIPTION' is empty (NaN or empty string)
merged_df = merged_df[merged_df['INTERVALSTART'].notna() & (merged_df['INTERVALSTART'] != '')]

# Convert the 'USCS' column to uppercase
merged_df['USCS'] = merged_df['USCS'].str.upper()

# Display the DataFrame
merged_df.head()

,Unnamed: 0,WCRNUMBER,DECIMALLAT,DECIMALLON,UTMX_y,UTMY_y,INTERVALSTART,INTERVALEND,MATERIALTYPE,MATERIALCOLOR,MATERIALTEXTURE,MATERIALDESCRIPTION,COMBINED_MATERIALS,KEYWORDS,USCS,HydraulicConductivity,AverageCoarseFraction,Avg Specific Yield (%),Avg Ss (1/L),Avg Kv (ft/day)
0,333,WCR2021-006252,35.0492,-118.98,866715.151022,3.885894e+06,0.0,370.0,Sand,Brown,Medium To Coarse,Sand/Gravel,Sand Medium To Coarse Sand/Gravel,"sand,sand,gravel",SW,4.510000,60.0,35.000,0.000230,0.5000
1,333,WCR2021-006252,35.0492,-118.98,866715.151022,3.885894e+06,370.0,880.0,Clayey Sand,Brown,Clayey,Clay/Sand/Rock,Clayey Sand Clayey Clay/Sand/Rock,"clayey sand,clay/sand,rock",SW,4.510000,60.0,35.000,0.000230,0.5000
2,333,WCR2021-006252,35.0492,-118.98,866715.151022,3.885894e+06,880.0,910.0,Gravel,Brown,Gravely,Rock/Gravel,Gravel Gravely Rock/Gravel,"gravel,rock,gravel",ROCK,0.000028,8.0,0.505,0.000001,0.0005
3,333,WCR2021-006252,35.0492,-118.98,866715.151022,3.885894e+06,910.0,1000.0,Clayey Gravel,Brown,Clayey,Clay/Rock,Clayey Gravel Clayey Clay/Rock,"gravel,clay,rock",CH,0.016402,2.5,2.000,0.003490,0.0005
4,333,WCR2021-006252,35.0492,-118.98,866715.151022,3.885894e+06,1000.0,1220.0,Clayey Gravel,Brown,Clayey,Clay/Gravel,Clayey Gravel Clayey Clay/Gravel,"gravel,clay,gravel",CH,0.016402,2.5,2.000,0.003490,0.0005


In [11]:
# Save the updated DataFrame to a new CSV file
output_file = '2-updated_geologiclog_quick-pick.csv'
merged_df.to_csv(output_file, index=False)

print(f"Updated CSV file saved as '{output_file}'")

Updated CSV file saved as '2-updated_geologiclog_quick-pick.csv'
